# Some common 'tricks'

When modelling problems in ASP, it turns out that there are some 'tricks' that come in handy a lot of the time. Here we'll run through some of the most common of these tricks.

Let's start with setting up a function to print answer sets of a program:

In [1]:
import clingo

def print_answer_sets(program):
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])
    control.configuration.solve.models = 0
    for model in control.solve(yield_=True):
        sorted_model = [str(atom) for atom in model.symbols(shown=True)]
        sorted_model.sort()
        print("Answer set: {{{}}}".format(", ".join(sorted_model)))

## Generating 

The following tricks are useful for generating the right search space.

### Generating assignments to binary variables

You can generate all assignments to a set of binary variables as follows (e.g., truth assignments).

In [2]:
print_answer_sets("""
    var(1..3).
    true(X) :- not false(X), var(X).
    false(X) :- not true(X), var(X).
    #show true/1.
    #show false/1.
""")

Answer set: {false(1), false(2), false(3)}
Answer set: {false(1), false(3), true(2)}
Answer set: {false(1), false(2), true(3)}
Answer set: {false(1), true(2), true(3)}
Answer set: {false(2), false(3), true(1)}
Answer set: {false(2), true(1), true(3)}
Answer set: {false(3), true(1), true(2)}
Answer set: {true(1), true(2), true(3)}


Or alternatively like this:

In [3]:
print_answer_sets("""
    var(1..3).
    1 { true(X); false(X) } 1 :- var(X).
    #show true/1.
    #show false/1.
""")

Answer set: {false(2), false(3), true(1)}
Answer set: {false(3), true(1), true(2)}
Answer set: {false(2), true(1), true(3)}
Answer set: {true(1), true(2), true(3)}
Answer set: {false(1), false(2), true(3)}
Answer set: {false(1), false(2), false(3)}
Answer set: {false(1), true(2), true(3)}
Answer set: {false(1), false(3), true(2)}


### Generating assignments to $n$-ary variables

Generating all assignments to variables with domains of more than two elements, you can do like this:

In [4]:
print_answer_sets("""
    var(1..2).
    value(1..4).
    assign(X,V) :- var(X), value(V), not assign(X,V') : value(V'), V' != V.
    #show assign/2.
""")

Answer set: {assign(1,1), assign(2,3)}
Answer set: {assign(1,1), assign(2,2)}
Answer set: {assign(1,1), assign(2,1)}
Answer set: {assign(1,1), assign(2,4)}
Answer set: {assign(1,4), assign(2,1)}
Answer set: {assign(1,2), assign(2,1)}
Answer set: {assign(1,3), assign(2,1)}
Answer set: {assign(1,2), assign(2,3)}
Answer set: {assign(1,2), assign(2,2)}
Answer set: {assign(1,2), assign(2,4)}
Answer set: {assign(1,4), assign(2,2)}
Answer set: {assign(1,3), assign(2,2)}
Answer set: {assign(1,4), assign(2,4)}
Answer set: {assign(1,4), assign(2,3)}
Answer set: {assign(1,3), assign(2,4)}
Answer set: {assign(1,3), assign(2,3)}


Or a bit more compactly/intuitively, like this:

In [5]:
print_answer_sets("""
    var(1..2).
    value(1..4).
    1 { assign(X,V) : value(V) } 1 :- var(X).
    #show assign/2.
""")

Answer set: {assign(1,3), assign(2,4)}
Answer set: {assign(1,3), assign(2,1)}
Answer set: {assign(1,3), assign(2,2)}
Answer set: {assign(1,3), assign(2,3)}
Answer set: {assign(1,4), assign(2,4)}
Answer set: {assign(1,4), assign(2,2)}
Answer set: {assign(1,4), assign(2,3)}
Answer set: {assign(1,4), assign(2,1)}
Answer set: {assign(1,2), assign(2,3)}
Answer set: {assign(1,2), assign(2,1)}
Answer set: {assign(1,2), assign(2,4)}
Answer set: {assign(1,2), assign(2,2)}
Answer set: {assign(1,1), assign(2,3)}
Answer set: {assign(1,1), assign(2,4)}
Answer set: {assign(1,1), assign(2,2)}
Answer set: {assign(1,1), assign(2,1)}


### Generating one-to-one assignments

If you have two sets of equal size, and you want to generate all one-to-one assignments between these sets, you can do that as follows:

In [6]:
print_answer_sets("""
    side1(1..3).
    side2(a;b;c).
    1 { match(S1,S2) : side1(S1) } 1 :- side2(S2).
    1 { match(S1,S2) : side2(S2) } 1 :- side1(S1).
    #show match/2.
""")

Answer set: {match(1,c), match(2,b), match(3,a)}
Answer set: {match(1,c), match(2,a), match(3,b)}
Answer set: {match(1,b), match(2,c), match(3,a)}
Answer set: {match(1,b), match(2,a), match(3,c)}
Answer set: {match(1,a), match(2,c), match(3,b)}
Answer set: {match(1,a), match(2,b), match(3,c)}


### Generating one-to-many assignments

If you have two sets $S_1$ and $S_2$ (possibly of different size), and you want to generate all assignments where every element in $S_1$ is assigned to exactly one element in $S_2$ (but not vice versa), then this is a way to do that:

In [7]:
print_answer_sets("""
    side1(1..2).
    side2(a;b;c).
    1 { match(S1,S2) : side2(S2) } 1 :- side1(S1).
    #show match/2.
""")

Answer set: {match(1,c), match(2,a)}
Answer set: {match(1,c), match(2,b)}
Answer set: {match(1,c), match(2,c)}
Answer set: {match(1,b), match(2,a)}
Answer set: {match(1,b), match(2,c)}
Answer set: {match(1,b), match(2,b)}
Answer set: {match(1,a), match(2,a)}
Answer set: {match(1,a), match(2,b)}
Answer set: {match(1,a), match(2,c)}


### Generating arbitrary assignments

If you have two sets $S_1$ and $S_2$ (possibly of different size), and you want to generate *all possible* (partial) assignments, you can do this:

In [8]:
print_answer_sets("""
    side1(1..2).
    side2(a;b).
    { match(S1,S2) : side1(S1) } :- side2(S2).
    #show match/2.
""")

Answer set: {}
Answer set: {match(1,b)}
Answer set: {match(2,a)}
Answer set: {match(1,b), match(2,a)}
Answer set: {match(2,b)}
Answer set: {match(2,a), match(2,b)}
Answer set: {match(1,b), match(2,b)}
Answer set: {match(1,b), match(2,a), match(2,b)}
Answer set: {match(1,a)}
Answer set: {match(1,a), match(1,b)}
Answer set: {match(1,a), match(2,b)}
Answer set: {match(1,a), match(1,b), match(2,b)}
Answer set: {match(1,a), match(2,a)}
Answer set: {match(1,a), match(2,a), match(2,b)}
Answer set: {match(1,a), match(1,b), match(2,a)}
Answer set: {match(1,a), match(1,b), match(2,a), match(2,b)}


### Generating injective assignments

If you have two sets $S_1$ and $S_2$ (possibly of different size), and you want to generate all assignments where every element in $S_1$ is assigned to exactly one element in $S_2$ (but not vice versa) that are *injective* (i.e., no two elements in $S_1$ are assigned to the same element of $S_2$), you can do that as follows:

In [9]:
print_answer_sets("""
    side1(1..2).
    side2(a;b;c).
    { match(S1,S2) : side1(S1) } 1 :- side2(S2).
    1 { match(S1,S2) : side2(S2) } 1 :- side1(S1).
    #show match/2.
""")

Answer set: {match(1,c), match(2,a)}
Answer set: {match(1,c), match(2,b)}
Answer set: {match(1,b), match(2,a)}
Answer set: {match(1,b), match(2,c)}
Answer set: {match(1,a), match(2,b)}
Answer set: {match(1,a), match(2,c)}


### Generating arbitrary subsets

Selecting an arbitrary subset of elements from a given set can be done as follows:

In [10]:
print_answer_sets("""
    element(a;b;c).
    { select(E) } :- element(E).
    #show select/1.
""")

Answer set: {}
Answer set: {select(b)}
Answer set: {select(c)}
Answer set: {select(b), select(c)}
Answer set: {select(a)}
Answer set: {select(a), select(c)}
Answer set: {select(a), select(b)}
Answer set: {select(a), select(b), select(c)}


Or alternatively like this:

In [11]:
print_answer_sets("""
    element(a;b;c).
    { select(E) : element(E) }.
    #show select/1.
""")

Answer set: {}
Answer set: {select(b)}
Answer set: {select(c)}
Answer set: {select(b), select(c)}
Answer set: {select(a)}
Answer set: {select(a), select(c)}
Answer set: {select(a), select(b)}
Answer set: {select(a), select(b), select(c)}


### Generating subsets of size $k$

If you want to generate all subsets that are of size *exactly* $k$, you can do this:

In [12]:
print_answer_sets("""
    element(a;b;c).
    2 { select(E) : element(E) } 2.
    #show select/1.
""")

Answer set: {select(b), select(c)}
Answer set: {select(a), select(c)}
Answer set: {select(a), select(b)}


### Generating subsets of size $\leq k$

If you want to generate all subsets that are of size *at most* $k$, you can do this:

In [13]:
print_answer_sets("""
    element(a;b;c).
    { select(E) : element(E) } 2.
    #show select/1.
""")

Answer set: {}
Answer set: {select(c)}
Answer set: {select(b)}
Answer set: {select(b), select(c)}
Answer set: {select(a)}
Answer set: {select(a), select(b)}
Answer set: {select(a), select(c)}


### Generating subsets of size $\geq k$

If you want to generate all subsets that are of size *at least* $k$, you can do this:

In [14]:
print_answer_sets("""
    element(a;b;c).
    2 { select(E) : element(E) }.
    #show select/1.
""")

Answer set: {select(b), select(c)}
Answer set: {select(a), select(c)}
Answer set: {select(a), select(b), select(c)}
Answer set: {select(a), select(b)}


## Constraints

The following tricks are useful for filtering out incorrect solutions, after you have generated a search space.

We will illustrate these with the example case where we generated an arbitrary subset of elements, but the same tricks apply also to the other cases.

### Basic constraints

If you want to ensure that `something` is **true**, you can add the constraint `:- not something.` (which can be read as: "It is not the case that `something` is not true").

For example, if you want to ensure that `a` is selected, you can use `:- not select(a).`.

In [15]:
print_answer_sets("""
    element(a;b;c).
    { select(E) } :- element(E).
    
    :- not select(a).
    
    #show select/1.
""")

Answer set: {select(a)}
Answer set: {select(a), select(c)}
Answer set: {select(a), select(b)}
Answer set: {select(a), select(b), select(c)}


If you want to ensure that `something` is **false**, you can add the constraint `:- something.` (which can be read as: "It is not the case that `something` is true").

For example, if you want to ensure that `a` is **not** selected, you can use `:- select(a).`.

In [16]:
print_answer_sets("""
    element(a;b;c).
    { select(E) } :- element(E).
    
    :- select(a).
    
    #show select/1.
""")

Answer set: {}
Answer set: {select(c)}
Answer set: {select(b)}
Answer set: {select(b), select(c)}


### AND-constraints

If you want to ensure that both `thing1` and `thing2` are true, you can define a new predicate (e.g., `my_property`), add rules that express that `my_property` is true if both `thing1` and `thing2` are true, and add a constraint that says that `my_property` must be true.

For example:

In [17]:
print_answer_sets("""
    element(a;b;c).
    { select(E) } :- element(E).
    
    my_property :- select(a), select(b).
    :- not my_property.
    
    #show select/1.
""")

Answer set: {select(a), select(b)}
Answer set: {select(a), select(b), select(c)}


This strategy (of defining a new predicate, defining when exactly this predicate is true, and requiring it to be true) works for more complicated cases as well.

In this simple example, we could easily have done without the new predicate too, e.g.:

In [18]:
print_answer_sets("""
    element(a;b;c).
    { select(E) } :- element(E).
    
    :- not select(a).
    :- not select(b).
    
    #show select/1.
""")

Answer set: {select(a), select(b)}
Answer set: {select(a), select(b), select(c)}


### OR-constraints

If you want to ensure that `thing1` is true **or** `thing2` is true, you can use the strategy of introducing a new predicate, like this:

In [19]:
print_answer_sets("""
    element(a;b;c).
    { select(E) } :- element(E).
    
    my_property :- select(a).
    my_property :- select(b).
    :- not my_property.
    
    #show select/1.
""")

Answer set: {select(b)}
Answer set: {select(b), select(c)}
Answer set: {select(a)}
Answer set: {select(a), select(c)}
Answer set: {select(a), select(b)}
Answer set: {select(a), select(b), select(c)}


Or you can add a constraint `:- not thing1, not thing2.` (which can be read as: "It is not the case that both `thing1` is false and `thing2` is false").

For example:

In [20]:
print_answer_sets("""
    element(a;b;c).
    { select(E) } :- element(E).
    
    :- not select(a), not select(b).
    
    #show select/1.
""")

Answer set: {select(b)}
Answer set: {select(b), select(c)}
Answer set: {select(a)}
Answer set: {select(a), select(c)}
Answer set: {select(a), select(b)}
Answer set: {select(a), select(b), select(c)}


### IMPLIES-constraints

If you want to express that if `thing1` is true, then also `thing2` must be true, you can do that like this, for example:

In [21]:
print_answer_sets("""
    element(a;b;c).
    { select(E) } :- element(E).
    
    :- select(a), not select(b).
    
    #show select/1.
""")

Answer set: {}
Answer set: {select(c)}
Answer set: {select(b)}
Answer set: {select(b), select(c)}
Answer set: {select(a), select(b)}
Answer set: {select(a), select(b), select(c)}


Again, also here the strategy of defining a new predicate would work well.

### IF-AND-ONLY-IF-constraints

If you want to express that two things must either both be true, or both be false, you can do that by using two 'if-then' constraints. For example like this:

In [22]:
print_answer_sets("""
    element(a;b;c).
    { select(E) } :- element(E).
    
    :- select(a), not select(b).
    :- not select(a), select(b).
    
    #show select/1.
""")

Answer set: {}
Answer set: {select(c)}
Answer set: {select(a), select(b)}
Answer set: {select(a), select(b), select(c)}
